## "journal pacifique" web bot
parses sources for new articles, translates them into target languages ["es","pt","pl","zh-CN"]. Wraps it up in a simple html document and writes it out to the appropriate directory.

#### TO DO:

*   resizable design
*   implement an automatic parser
*   mobile-friendly design
*   detect and highlight keywords
*   automatic git push
*   run the script on an aws server
*   maybe a language rating mechanism?
*   don't send a separate request to the website for every translation (__done__)
*   make sure same pages aren't crawled twice. Even if they are crawled twice they shouldn't be overwritten.
*   subscribe and social media block

translations under a certain quality are dispensable

#### Requests count as of 12.09.19


*   Google Translate reqs: 4 * language * article. (now L * A)
    *   SHOULD BE REDUCED! translate articles by seperating them with simple tags.
    *   this should reduce GT reqs to language * article  (__done__)
*   Website reqs: number of articles
*   Total: ( 4 * L + 1) * A -> ( L + 1 ) * A

NOTE: google transalte tokens refresh every hour (?)






---



In [1]:
#for colab
#!pip install googletrans
#!pip install mechanize

## Character limit: 15K

In [2]:
import googletrans
from googletrans import Translator
import re

translator = Translator()

In [3]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [4]:
def string_strip(s):
    return re.sub(r"[^A-Za-z0-9]", "-", s)

In [5]:
#def sciencedaily_parse_links():
  

In [6]:
from urllib.request import Request, urlopen

def sciencedaily_parse_article(url):
  
    site = url
    html_string = ""
    tagged_w_ps = ""

    # parse html

    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    html_string = webpage.decode("utf-8") 
  
    # get info
  
    # get the headline

    headline = find_between(html_string, '<h1 id="headline" class="headline">','</h1>')
  
    # get the meta
  
    meta     = find_between(html_string, '<dd id="abstract">','</dd>')
  
    # get the img address
  
    imgaddr  = find_between(html_string, 'center-block" src="','"')
    if(imgaddr!=""):
        imgaddr  = 'https://www.sciencedaily.com'+imgaddr
    else:
        imgaddr  = '../../../dummy.jpeg'
  
    # get the img alt
  
    imgalt   = find_between(html_string, '<div class="photo-caption">','</div>')
    cred     = '\t'+find_between(html_string, '<div class="photo-credit"><em>','</div>')
  
    # get the citation
  
    citation = find_between(html_string, '<div role="tabpanel" class="tab-pane active" id="citation_mla">','</div>')
  
    
    # get the article itself

    article  = find_between(html_string, '<div id="text">','</div>')
    article  = re.sub(r"<p>" ,     "", article)
    article  = re.sub(r"</p>", "\n", article)
  
    # get the href at the end if it exists
    href_tab = ""
    href_tab = href_tab + find_between(article, '<a href','</a>')
  
    # delete the href and useless tags from the article
  
    article  = re.sub(r"<a href.*?</a>", "", article)
    article  = re.sub(r"<.*?>", "",          article)
    article  = re.sub(r" -- ", ", ",         article)
  
    # special characters, this list will expand.
  
    #article  = re.sub(r"&uuml;", "ü",         article)
    #article  = re.sub(r"&ouml;", "ö",         article)
    #article  = re.sub(r"&eacute;", "e",         article)
    # new sltn:
    import html
    article   = html.unescape(article)
      
            #y                    #y                                #y                                                       #y                  
    return {"headline": headline, "meta": meta, "imgaddr": imgaddr, "imgalt": imgalt, "imgcredit":cred,"citation": citation, "article": article, "href": href_tab}
  

In [7]:
def get_translation(article_dictionary,target_language):
    big_string= ""
    translations = {}
    token_in = "(#@)"
    if (target_language=="zh-CN"):
      token_out = "（＃@）"
    else: token_out="(# @)"
      
    
    #sırası karışmasın!! keylerden array oluştur, arrayi itere et.
    keys = ["headline","meta","article","imgalt"]
    
    for key in keys:
        print(key)
        if big_string != "":
            big_string = big_string + token_in + article_dictionary[key]
        else:
            big_string = article_dictionary[key]
            
    big_translated_string = translator.translate(big_string,dest=target_language).text
    
    translations_array    = big_translated_string.split(token_out)
   
    #print(big_translated_string)
    #print(len(translations_array),translations_array)
    
    for i in range (4):
        translations[keys[i]]=translations_array[i]

    return translations

In [8]:
;# deprecated
def get_language_dictionary(target_language):
    ld = {}
    words = ["journal pacifique", "homepage", "archive", "about us", "source:"]
    for word in words:
        print(word)
        ld[word] = translator.translate(word, dest=target_language).text
    return ld

In [9]:
def html_from_dictionary(vanilla_dictionary, translated_dictionary, target_language, language_dictionary): # translated & language could be just one dictionary this was stupid.
  
    from datetime import date
    hoy  = date.today()
    d1   = hoy.strftime("%d/%m/%Y")
  
    template  ="https://journalpacifique.com/jp.temp"
    req       = Request(template, headers={'User-Agent': 'Mozilla/5.0'})
    webpage   = urlopen(req).read()

    html = webpage.decode("utf-8") 
  
    html_article = re.sub(r"\n\n", "</p>\n\n<p>",   translated_dictionary["article"])
    html_article = "<p>" + html_article + "</p>"
  
    print(translated_dictionary)
    print(language_dictionary)
  
    html = re.sub(r"\$\$article-title%%",  translated_dictionary["headline"],         html)
    html = re.sub(r"\$\$img-alt%%",        translated_dictionary["imgalt"],           html)
    html = re.sub(r"\$\$article-meta%%",   translated_dictionary["meta"],             html)
    html = re.sub(r"\$\$source%%",         vanilla_dictionary["citation"],            html)
    html = re.sub(r"\$\$img.jpg%%",        vanilla_dictionary["imgaddr"],             html)
    html = re.sub(r"\$\$article-text%%",   html_article                  ,            html)
  
    html = re.sub(r"\$\$home%%",           language_dictionary["homepage-"+target_language],           html)
    html = re.sub(r"\$\$archive%%",        language_dictionary["archive-"+target_language],            html)
    html = re.sub(r"\$\$about%%",          language_dictionary["about us-"+target_language],           html)
    html = re.sub(r"\$\$jp-translation%%", language_dictionary["journal pacifique-"+target_language],  html)
    html = re.sub(r"\$\$taken-from%%",     language_dictionary["source:-"+target_language],            html)
    html = re.sub(r"\$\$target-languages%%",     target_language,            html)
  
    html = re.sub(r"\$\$imgcredit%%",       vanilla_dictionary["imgcredit"],          html)
  
    html = re.sub(r"<a",       '<a style="color:black"',          html)
  
  
  
    html = re.sub(r"\$\$href%%",           vanilla_dictionary["href"],            html)
  
    html = re.sub(r"-- ", ",",  html)
 
    html = re.sub(r"\$\$date%%", d1, html)
  
    #html = re.sub(r'</head>', '<style> p { text-indent: 30px;} </style>\n\t</head>', html)
  
    return html
  

In [10]:
def getnewpath(dic,translated_dict):
  
    from datetime import date
    hoy  = date.today()
    date = hoy.strftime("%Y-%m-%d")
    
    translated_dict["date"]=date
  
    urlheadline = string_strip(dic["headline"])+"-"+target_language
    newpathaddr = target_language+"/posts/"+date+"/"
    newheadline = newpathaddr+urlheadline+".html"
    
    translated_dict["path"]=newheadline
    
    # this was implemented for short headlines (~10char)
    #if newheadline in headlinessofar:
    #    c=2
    #    while (newheadline[:-5]+repr(c)+newheadline[-5:] in headlinessofar):
    #        c+=1
    #    newheadline = newheadline[:-5]+repr(c)+newheadline[-5:]
    
    headlinessofar.append(newheadline)
    return [newheadline,newpathaddr,urlheadline]

In [11]:
def dic_to_dirfile(dic,target_language, elements_dictionary):
    
    #we used to translate all elements at every step, now it is automated due to
    #google translate's quota restrictions.
    
    #zh_ld = get_language_dictionary(target_language)

    tr_di = get_translation(dic,target_language)
    tr_di["lang"]=target_language
    tr_di["imgaddr"]=dic["imgaddr"]
    
    htmlx = html_from_dictionary(dic, tr_di, target_language, elements_dictionary)
  
    newdirs = getnewpath(dic,tr_di)
  
    import os
    cmd = "mkdir -p "+newdirs[1]
    os.system(cmd)
  
    f= open(newdirs[0],"w+")
    f.write(htmlx)
  
    return [htmlx,tr_di,newdirs[2]]

In [12]:
elements_dictionary = {
    "journal pacifique-es":"periódico pacífico", 
    "homepage-es":"página principal", 
    "archive-es":"archivo",
    "about us-es":"sobre nosotros",
    "source:-es":"fuente:",
    "journal pacifique-pt":"jornal pacífico", 
    "homepage-pt":"pagina inicial", 
    "archive-pt":"arquivo",
    "about us-pt":"sobre nós",
    "source:-pt":"fonte:",
    "journal pacifique-zh-CN":"和平的报纸", 
    "homepage-zh-CN":"主页", 
    "archive-zh-CN":"档案",
    "about us-zh-CN":"关于我们",
    "source:-zh-CN":"资源",
    "journal pacifique-pl":"spokojna gazeta", 
    "homepage-pl":"strona główna", 
    "archive-pl":"archiwum",
    "about us-pl":"o nas",
    "source:-pl":"źródło",
    "latest-articles-es":"Últimos artículos",
    "latest-articles-pt":"Artigos Mais Recentes",
    "latest-articles-pl":"Ostatnie artykuły",
    "latest-articles-zh-CN":"最新的文章",
    "hometext-es":"Bienvenido al periódico pacífico, donde tenemos alfombras y recuerdos.",
    "hometext-pt":"bem-vindo ao jornal pacífico, temos tapetes e lembranças.",
    "hometext-pl":"witamy w spokojnej gazecie, która ma dywany i pamiątki.",
    "hometext-zh-CN":"欢迎来到宁静的报纸，那里有地毯和纪念品。",
}

In [13]:
def url_to_dirfile(url,target_language):

    ar_di = sciencedaily_parse_article(url)

    htmlx = dic_to_dirfile(ar_di,target_language, elements_dictionary)
  
    return [htmlx,ar_di]

In [14]:
articles = ["https://www.sciencedaily.com/releases/2019/09/190910080021.htm",
            "https://www.sciencedaily.com/releases/2019/09/190911174649.htm",
            "https://www.sciencedaily.com/releases/2019/09/190911142815.htm",
            "https://www.sciencedaily.com/releases/2019/09/190911142740.htm",
            "https://www.sciencedaily.com/releases/2019/09/190911142736.htm",
            "https://www.sciencedaily.com/releases/2019/09/190911121952.htm",
            "https://www.sciencedaily.com/releases/2019/09/190904125337.htm",
            "https://www.sciencedaily.com/releases/2019/09/190903084035.htm",
            "https://www.sciencedaily.com/releases/2019/08/190829115427.htm",
           ]   
            
languages = ["es","pt","pl","zh-CN"]

In [15]:
headlinessofar = []
articlessofar  = {}

In [16]:
for article in articles:
    dic = sciencedaily_parse_article(article)
    for target_language in languages:
            tmp = dic_to_dirfile(dic,target_language, elements_dictionary)
            articlessofar[tmp[2]]=tmp[1]

headline
meta
article
imgalt
{'meta': '. Una especie de pterosaurio recientemente identificada se encuentra entre los animales voladores más grandes de la historia.\n', 'imgalt': ' Ilustración de pterosaurios en vuelo (imagen de stock).', 'lang': 'es', 'imgaddr': 'https://www.sciencedaily.com/images/2019/09/190910080021_1_540x360.jpg', 'article': '\nCryodrakon boreas, del grupo de pterosaurios Azhdarchid (a menudo incorrectamente llamados \'pterodáctilos\'), era un reptil volador con una envergadura de hasta 10 metros que vivió durante el período Cretácico hace unos 77 millones de años.\n\nSus restos fueron descubiertos hace 30 años en Alberta, Canadá, pero los paleontólogos habían asumido que pertenecían a una especie de pterosaurio ya conocida descubierta en Texas, EE. UU., Llamada Quetzalcoatlus.\n\nEl estudio, publicado en el Journal of Vertebrate Paleontology, revela que en realidad es una nueva especie y el primer pterosaurio descubierto en Canadá.\n\nEl Dr. David Hone, autor pri

{'meta': '. Według nowych badań nowo zidentyfikowany gatunek pterozaura jest jednym z największych kiedykolwiek latających zwierząt.\n', 'imgalt': ' Ilustracja pterozaurów w locie (obraz czasowy).', 'lang': 'pl', 'imgaddr': 'https://www.sciencedaily.com/images/2019/09/190910080021_1_540x360.jpg', 'article': '\nCryodrakon boreas, z grupy pterozaurów z Azhdarchid (często błędnie zwanych „pterodaktylami”), był latającym gadem o rozpiętości skrzydeł do 10 metrów, który żył w okresie kredowym około 77 milionów lat temu.\n\nJego szczątki zostały odkryte 30 lat temu w Albercie w Kanadzie, ale paleontolodzy założyli, że należą do znanego już gatunku pterozaura odkrytego w Teksasie w USA, zwanego Quetzalcoatlus.\n\nBadanie, opublikowane w Journal of Vertebrate Paleontology, ujawnia, że \u200b\u200bjest to właściwie nowy gatunek i pierwszy pterozaur odkryty w Kanadzie.\n\nDr David Hone, główny autor badania z Queen Mary University of London, powiedział: „To fajne odkrycie, wiedzieliśmy, że to zw

{'meta': '. Durante anos, os cientistas exploraram maneiras de alterar as células dos microorganismos nos esforços para melhorar a quantidade de produtos fabricados, incluindo medicamentos, combustíveis e até cerveja. Mas alterar os processos genéticos e regulatórios que ocorrem dentro das células apresenta desafios. Agora, os bioengenheiros estão trabalhando com uma equipe de pesquisadores para projetar consórcios microbianos, em que as subpopulações de células são projetadas para trabalharem em conjunto para realizar a função desejada.\n', 'imgalt': '', 'lang': 'pt', 'imgaddr': '../../../dummy.jpeg', 'article': '\nPorém, alterar os processos genéticos e regulatórios que ocorrem dentro das células apresenta desafios.\n\nPara começar, as células já estão programadas para realizar seus processos cotidianos normais com a máxima eficiência; quaisquer alterações feitas pelos engenheiros para aumentar a produção de uma determinada substância por uma célula podem, por sua vez, perturbar esse

{'meta': '多年来，科学家们一直在探索改变微生物细胞的方法，以改善生产多少产品，包括药品，燃料，甚至啤酒。但改变细胞内发生的遗传和调节过程带来了挑战。现在，生物工程师正在与一组研究人员合作设计微生物聚生体，其中细胞亚群被设计成一起工作以实现所需的功能。\n', 'imgalt': '', 'lang': 'zh-CN', 'imgaddr': '../../../dummy.jpeg', 'article': '\n但是，改变细胞内发生的遗传和调节过程带来了挑战。\n\n首先，细胞已经被编程，以最高效率执行正常的日常过程;工程师为增加细胞产生某种物质所做的任何改变反过来会破坏这些过程并使细胞负担过重。\n\n为了解决这个问题，马里兰大学生物工程教授兼大学Robert E. Fischell生物医学设备研究所所长William E. Bentley正在与一组研究人员合作，专注于工程微生物聚生体，其中细胞亚群设计为共同努力，实现理想的功能。该战略，该领域的其他人也在探索，允许工程师设计专门的单元格，并在一组单元格中分配目标工作负载。权衡是驱动财团执行一系列特定任务需要工程师以某种方式调节每个细胞亚群的存在量。\n\n尽管面临这一挑战，但很少有科学研究专注于开发可以调节联盟内细胞亚群组成的装置或系统。大多数情况下，此类研究要求工程师依靠手动或用户控制的技术在细胞亚群之间取得平衡。\n\n因此，Bentley和他的团队意识到更强大的方法是重新设计细胞本身以自主地协调其亚群密度。他们的技术在今天发表的Nature Communications论文中得到了强调。\n\n与该领域的其他人一样，Bentley及其生物分子和代谢工程实验室的成员之前曾调查群体感应（QS），一种细胞形式的细胞间通信，以设计细菌菌株之间和之间的通信电路，以协调其行为。 QS涉及小信号分子的合成，称为自诱导物（AI），其由个体细菌分泌但用于协调它们的反应。一旦AI水平达到阈值，发出“法定数量”的细胞信号，AI信号在细胞内转运，在那里它们激活基因表达并实现协调反应。\n\n但是，宾利和他的团队更进了一步。该小组开发了第一个已知的平台，用于基于存在称为自诱导剂-2（AI-2）的环境线索，对聚生体组成进行自主和有针对性的调节。 AI-2被认为是一种通用的QS信号，这意味着它被许多种细菌识别和产生。它广泛地

{'meta': ' Os astrônomos descobriram uma das maiores características já observadas no centro da Via Láctea: um par de enormes bolhas emissoras de rádio que elevam centenas de anos-luz acima e abaixo da região central da nossa galáxia.\n', 'imgalt': ' Via Láctea (imagem).', 'lang': 'pt', 'imgaddr': 'https://www.sciencedaily.com/images/2019/09/190911142815_1_540x360.jpg', 'article': '\nUsando o telescópio MeerKAT do Observatório de Radioastronomia da África do Sul (SARAO), Ian Heywood, do Departamento de Física da Universidade de Oxford, e seus colegas mapearam amplas regiões no centro da galáxia, realizando observações em comprimentos de onda próximos a 23 centímetros. Emissões de rádio desse tipo são geradas em um processo conhecido como radiação síncrotron, em que elétrons flutuantes são acelerados à medida que interagem com poderosos campos magnéticos. Isso produz um sinal de rádio característico que pode ser usado para rastrear regiões energéticas no espaço. A luz do rádio vista pel

...

{'meta': ' O Tyrannosaurus rex, um dos maiores dinossauros carnívoros do planeta, tinha um ar condicionado na cabeça, sugerem cientistas que estão desafiando mais de um século antes crenças.\n', 'imgalt': '\n<em> T. rex </em> esqueleto, cabeça e pescoço (imagem).', 'lang': 'pt', 'imgaddr': 'https://www.sciencedaily.com/images/2019/09/190904125337_1_540x360.jpg', 'article': '\nNo passado, os cientistas acreditavam que dois grandes buracos no teto do crânio de um T. rex, chamados fenestra dorsotemporal, estavam cheios de músculos que ajudam nos movimentos da mandíbula.\n\nMas essa afirmação intrigou Casey Holliday, professora de anatomia da MU School of Medicine e principal pesquisadora do estudo.\n\n"É realmente estranho que um músculo suba da mandíbula, faça uma curva de 90 graus e vá ao longo do teto do crânio", disse Holliday. "No entanto, agora temos muitas evidências convincentes para vasos sanguíneos nessa área, com base em nosso trabalho com jacarés e outros répteis".\n\nUsando i

headline
meta
article
imgalt
{'meta': ' Un reactor de electrocatálisis construido en la Universidad de Rice recicla dióxido de carbono para producir soluciones de combustible líquido puro usando electricidad. Los científicos detrás de la invención esperan que se convierta en una forma eficiente y rentable de reutilizar el gas de efecto invernadero y mantenerlo fuera de la atmósfera.\n', 'imgalt': ' Modelos de moléculas de dióxido de carbono (imagen de stock).', 'lang': 'es', 'imgaddr': 'https://www.sciencedaily.com/images/2019/09/190903084035_1_540x360.jpg', 'article': '\nEl reactor catalítico desarrollado por el laboratorio de ingeniería química y biomolecular de la Universidad de Rice, Haotian Wang, utiliza dióxido de carbono como materia prima y, en su último prototipo, produce altas concentraciones de ácido fórmico altamente purificado.\n\nEl ácido fórmico producido por los dispositivos tradicionales de dióxido de carbono necesita pasos de purificación costosos e intensivos en ener

{'meta': " Reaktor elektrokatalizy zbudowany na Uniwersytecie Rice'a przetwarza dwutlenek węgla, aby wytwarzać czyste roztwory paliw ciekłych przy użyciu energii elektrycznej. Naukowcy stojący za wynalazkiem mają nadzieję, że stanie się skutecznym i opłacalnym sposobem na ponowne wykorzystanie gazu cieplarnianego i utrzymanie go poza atmosferą.\n", 'imgalt': ' Modele cząsteczek dwutlenku węgla (zdjęcie stockowe).', 'lang': 'pl', 'imgaddr': 'https://www.sciencedaily.com/images/2019/09/190903084035_1_540x360.jpg', 'article': '\nReaktor katalityczny opracowany przez laboratorium inżyniera chemicznego i biomolekularnego na Uniwersytecie Rice w Haotian Wang wykorzystuje dwutlenek węgla jako surowiec, aw swoim najnowszym prototypie wytwarza wysoko oczyszczone i wysokie stężenia kwasu mrówkowego.\n\nWang powiedział, że kwas mrówkowy wytwarzany przez tradycyjne urządzenia do dwutlenku węgla wymaga kosztownych i energochłonnych etapów oczyszczania. Bezpośrednia produkcja czystych roztworów kwas

headline
meta
article
imgalt
{'meta': ' Al analizar los dientes fosilizados de algunos de nuestros antepasados \u200b\u200bmás antiguos, los científicos descubrieron que los primeros humanos amamantaron significativamente a sus bebés durante períodos más largos que sus parientes contemporáneos.\n', 'imgalt': ' Elemento de calcio en la tabla periódica (imagen de stock).', 'lang': 'es', 'imgaddr': 'https://www.sciencedaily.com/images/2019/08/190829115427_1_540x360.jpg', 'article': '\nLos resultados, publicados en la revista Science Advances, proporcionan una primera visión de la práctica del destete que de otro modo no se vería en el registro fósil.\n\nEl equipo probó cantidades mínimas de casi 40 dientes fosilizados de nuestros parientes fósiles sudafricanos, Homo temprano, Paranthropus robustus y Australopithecus africanus.\n\nMidieron las proporciones de sus isótopos de calcio estables en el esmalte dental, que son una función de la ingesta de leche materna de los bebés.\n\nAl reconst

{'meta': ' Analizując skamieniałe zęby niektórych z naszych najstarszych przodków, naukowcy odkryli, że pierwsi ludzie znacznie karmili swoje niemowlęta przez dłuższy czas niż ich współczesni krewni.\n', 'imgalt': ' Pierwiastek wapnia w układzie okresowym pierwiastków (obraz stockowy).', 'lang': 'pl', 'imgaddr': 'https://www.sciencedaily.com/images/2019/08/190829115427_1_540x360.jpg', 'article': "\nWyniki, opublikowane w czasopiśmie Science Advances, dają pierwszy wgląd w praktykę odsadzania, która w przeciwnym razie pozostaje niewidoczna w zapisie kopalnym.\n\nZespół pobrał próbki minut z prawie 40 skamieniałych zębów naszych południowoafrykańskich krewnych kopalnych, wczesnego Homo, Paranthropus robustus i Australopithecus africanus.\n\nZmierzyli proporcje stabilnych izotopów wapnia w szkliwie zębów, które są funkcją spożycia mleka matki przez niemowlęta.\n\nRekonstruując wiek rozwoju szkliwa zębów, pokazują, że wczesne potomstwo Homo było karmione piersią w znacznych proporcjach aż 

In [17]:
#get most recent 9 articles and get their "keys"

def form_index(target_language):
  
    homepage_text="welcome my friend we have carpets."
 
    jp_index = "https://journalpacifique.com/jp-index.temp"
    req = Request(jp_index, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    jp_index_html_string = webpage.decode("utf-8") 
    
    html = jp_index_html_string
    
    from datetime import date
    hoy  = date.today()
    d1   = hoy.strftime("%d/%m/%Y")
    
    html = re.sub(r"\$\$date%%", d1, html)

    html = re.sub(r"\$\$home%%",           elements_dictionary["homepage-"+target_language],           html)
    html = re.sub(r"\$\$archive%%",        elements_dictionary["archive-"+target_language],            html)
    html = re.sub(r"\$\$about%%",          elements_dictionary["about us-"+target_language],           html)
    html = re.sub(r"\$\$jp-translation%%", elements_dictionary["journal pacifique-"+target_language],  html)
    html = re.sub(r"\$\$latest-articles%%",elements_dictionary["latest-articles-"+target_language],    html)
    
    #save these to elements dictionary
    html = re.sub(r"\$\$homepage-text%%",  elements_dictionary["hometext-"+target_language],    html)
    
    html = re.sub(r"\$\$target-language%%",target_language,  html)

    asfl = []
    for key in articlessofar:
        if(articlessofar[key]["lang"]==target_language):
            asfl.append(key)
    
    for i in range (9):
        j=i+1
        key = asfl[i]
    
        html = re.sub(r"\$\$headline"+repr(j)+"%%",           articlessofar[key]["headline"],           html)
        html = re.sub(r"\$\$headlinemeta"+repr(j)+"%%",           articlessofar[key]["meta"],           html)
        if(articlessofar[key]["imgaddr"][:2]!=".."):
            html = re.sub(r"\$\$headline-img"+repr(j)+"%%",      articlessofar[key]["imgaddr"],         html)
        else:
            html = re.sub(r"\$\$headline-img"+repr(j)+"%%",      articlessofar[key]["imgaddr"][6:],              html)
    
    return html


In [18]:
def get_article_links_from_sd():
    
    sd_tech = "https://www.sciencedaily.com/news/top/technology/"
    req = Request(sd_tech, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    sd_tech_html_string = webpage.decode("utf-8") 
    
    find_between(html_string, '<h1 id="headline" class="headline">','</h1>')

In [19]:
def refresh_indices():
    for language in languages:

        htmlx = form_index(language)
        path  = language +"/index.html"

        f= open(path,"w+")
        f.write(htmlx)

In [20]:
refresh_indices()

In [21]:
"../,,/book.jpg"[6:]

'book.jpg'

In [47]:
articlessofar[1]

KeyError: 1

In [0]:
def form_index():


In [0]:
"""#parse given url (for quanta mag) and return translation of the article.

def quanta_parser(url, target_language='tr'):
  site = url
  html_string = ""
  tagged_w_ps = ""

  # parse html

  from mechanize import Browser
  br = Browser()
  br.set_handle_robots(False) #illegal
  br.open(site)
  html = br.response().readlines()
  for line in html:
    html_string = html_string+line.decode("utf-8") 
    
    
  # get text in the main section under <section> tag.  
  
  for item in html_string.split("</section>"):
    if "<section>" in item:
      tagged_w_ps = tagged_w_ps + item[ item.find("<section>")+len("<section>") : ]
      
  
  # get text in the paragraph tags (<p>).
  
  for item in html_string.split("</p>"):
    if "<p>" in item:
      tagged_w_ps = tagged_w_ps + (item[ item.find("<p>")+len("<p>") : ]+"ğ")
      
      
  # remove alt texts.
  tagged_w_ps = re.sub(r" *\..*(Quanta Magazine)", ".", tagged_w_ps)
      
  
  # take link tags out of the way and edit custom characters.
  # TODO: expand this list for future articles with other unicode characters.
      
  newstr = re.sub(r"(<a href).*?(\">)", "", tagged_w_ps)
  newstr = re.sub(r"(\\u2018)", "\"", newstr)
  newstr = re.sub(r"(\\u2019)", "\"", newstr)
  newstr = re.sub(r"(\xa0)", " ", newstr)
  newstr = re.sub(r"(&#8217)", "\'", newstr)
  newstr = re.sub(r"(</a>)", "", newstr)
  newstr = re.sub(r"\.<.*?(quote_attribution).*?<p>", " says ", newstr)
  newstr = re.sub(r"(<p>)", "", newstr)
  newstr = re.sub(r"(<).*(>)", ".", newstr)
  newstr = re.sub(r"(\",\"caption).*}}", "", newstr)
  
  
  # convert custom newline tag to newline
  newstr = re.sub(r"ğ+", "\n\n", newstr)
  
  # translate the text to your target language.
  
  translation = translator.translate(newstr, dest=target_language).text
  
  return translation
  
  """